In [1]:
!pip install tabulate > /dev/null
!pip install dill > /dev/null
from pydal import DAL, Field
from attrs import define, field, asdict


In [2]:

import os

server = os.getenv('POSTGRES_SERVER', '10.221.142.201')
db = DAL(f'postgres://username:password@{server}/db')
db.define_table('item',
    Field('gid','string'),
    Field('value','text')
)

<Table item (id, gid, value)>

In [150]:
import importlib
import edwh.core.pgcache # force a normal import or nop for a re-import
edwh.core.pgcache = importlib.reload(edwh.core.pgcache) # for a reload while loaded  (requires previous import)
# just import what we need.
from edwh.core.pgcache import define_model, cache_ids_that_depend_on, cache_ids_which_this_id_depends_on, cached, CachedResult, cache_hook_table_with_gid, DillableAttrsClass, debug_cache_dependencies, show_rows
show = show_rows

In [124]:
define_model(db)
db(db.cache).delete()
db(db.deps).delete()
db.commit()

In [125]:
db.rollback()

a = db.cache.insert(gid='a',value='aa')
b = db.cache.insert(gid='b',value='aabb')
db.deps.insert(cache_id=b,depends_on=a)
c = db.cache.insert(gid='c',value='aaabbbccc')
db.deps.insert(cache_id=c,depends_on=b)
d = db.cache.insert(gid='d',value='bbdd')
db.deps.insert(cache_id=d,depends_on=b)
db.commit()

In [126]:

show('cache',db(db.cache).select())
show('deps',db(db.deps).select())

---------[ cache ]----------
|   id | gid   | value     |
|------|-------|-----------|
|  179 | a     | aa        |
|  180 | b     | aabb      |
|  181 | c     | aaabbbccc |
|  182 | d     | bbdd      |

--------------[ deps ]--------------
|   id |   cache_id |   depends_on |
|------|------------|--------------|
|  244 |        180 |          179 |
|  245 |        181 |          180 |
|  246 |        182 |          180 |



In [127]:
db.rollback()
for row in db(db.cache).select():
    show(f'{row.gid}.requires',
         db(db.cache.id.belongs([r.cache_id for r in cache_ids_which_this_id_depends_on(db, row.id)])).select())

[ a.requires ]


------[ b.requires ]------
|   id | gid   | value   |
|------|-------|---------|
|  179 | a     | aa      |
|  180 | b     | aabb    |

-------[ c.requires ]-------
|   id | gid   | value     |
|------|-------|-----------|
|  179 | a     | aa        |
|  180 | b     | aabb      |
|  181 | c     | aaabbbccc |

------[ d.requires ]------
|   id | gid   | value   |
|------|-------|---------|
|  179 | a     | aa      |
|  180 | b     | aabb    |
|  182 | d     | bbdd    |



In [128]:
db.rollback()
for row in db(db.cache).select():
    show(f'{row.gid}.required_for',
         db(db.cache.id.belongs([r.cache_id for r in cache_ids_that_depend_on(db, row.id)])).select())

-----[ a.required_for ]-----
|   id | gid   | value     |
|------|-------|-----------|
|  179 | a     | aa        |
|  180 | b     | aabb      |
|  181 | c     | aaabbbccc |
|  182 | d     | bbdd      |

-----[ b.required_for ]-----
|   id | gid   | value     |
|------|-------|-----------|
|  180 | b     | aabb      |
|  181 | c     | aaabbbccc |
|  182 | d     | bbdd      |

[ c.required_for ]


[ d.required_for ]




In [129]:
@cached(db, key='times2-{0}')
def times2value(gid):
    value = db.cache(gid=gid).value
    result = value * 2
    return CachedResult(result, (gid,))

assert times2value('a').value == 'aaaa'
assert times2value('a').value == 'aaaa'
assert times2value('d').value == 'bbddbbdd'

In [130]:
show('cache',db(db.cache).select())
show('deps',db(db.deps).select())

-----------[ cache ]-----------
|   id | gid      | value     |
|------|----------|-----------|
|  179 | a        | aa        |
|  180 | b        | aabb      |
|  181 | c        | aaabbbccc |
|  182 | d        | bbdd      |
|  183 | times2-a | aaaa      |
|  184 | times2-d | bbddbbdd  |

--------------[ deps ]--------------
|   id |   cache_id |   depends_on |
|------|------------|--------------|
|  244 |        180 |          179 |
|  245 |        181 |          180 |
|  246 |        182 |          180 |
|  247 |        183 |          179 |
|  248 |        184 |          182 |



In [131]:
db.commit()

Onderstaande werkte wel in single-threaded modus,
maar met verschillende threads loopt alles in de soep blijkbaar.
https://realpython.com/intro-to-python-threading/
https://github.com/gonzalo123/pglistener/blob/main/pg_listener/pg_listener.py


In [132]:

# import select
# import threading
# from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
# messagebus = []
# def notified(notification):
#     messagebus.append(notification)
#     print(f'received on {notification.channel:>20}: {notification.payload!r}')
#
# def listen(db_uri, channel):
#     print('listen started. ')
#     db = DAL(db_uri)
#     conn = db._adapter.connection
#     conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
#     db.executesql(f'listen {channel};')
#     # db.executesql('select pg_listening_channels();')
#     while True:
#         print('selecting')
#         if select.select([conn], [], [], 5)[0]:
#             db.commit()
#             print('polling', messagebus )
#             conn.poll()
#             while conn.notifies:
#                 notify = conn.notifies.pop()
#                 print(threading.Thread(target=notified, args=(notify,)))
#
# print(threading.active_count())
# threading.Thread(target=listen, args=(db._uri, 'bla'))
# import time
# time.sleep(0.600)
# print(threading.active_count())
# listen(db._uri, 'bla')

In [133]:
# print(db.executesql("select pg_notify('bla','test');"))
# db.commit()
# print(messagebus)

In [134]:
import uuid
db.define_table('item',
    Field('gid','string',default=uuid.uuid4),
    Field('title','string'),
    Field('body','string'),
    redefine=True
)

<Table item (id, gid, title, body)>

In [135]:
cache_hook_table_with_gid(db, 'item')

In [136]:
db(db.item).delete()
db(db.cache).delete()
db(db.deps).delete()
for x in range(5):
    db.item.insert(gid=f'g{x}', title=f'title {x}', body=' '.join([str(_) for _ in [x,x,x,x,x]]))
db.commit()

In [137]:
show('item',db(db.item).select())
show('cache',db(db.cache).select())
show('deps',db(db.deps).select())

---------------[ item ]---------------
|   id | gid   | title   | body      |
|------|-------|---------|-----------|
|   31 | g0    | title 0 | 0 0 0 0 0 |
|   32 | g1    | title 1 | 1 1 1 1 1 |
|   33 | g2    | title 2 | 2 2 2 2 2 |
|   34 | g3    | title 3 | 3 3 3 3 3 |
|   35 | g4    | title 4 | 4 4 4 4 4 |

[ cache ]


[ deps ]




In [138]:



@define
class Item(DillableAttrsClass):
    id:int
    gid:str
    title:str
    body:str

@cached(db,
        key='{}',
        todb=Item.todb,
        fromdb=Item.fromdb
        )
def item(gid):
    value = Item(**db.item(gid=gid).as_dict())
    return CachedResult(value=value, required_gids=(gid,))


@cached(db,
        key=lambda gid:f'teaser-{gid}',
        todb=lambda teaser:teaser,
        fromdb=lambda teaser:teaser
)
def item_teaser(gid):
    record = item_teaser.track(item(gid))
    short = record.body[:5]+'...'
    result = f'{record.title}: {short}'
    return result

print(item_teaser('g0'))
print(item_teaser('g2'))

@cached(db, key=lambda gid:f'tile-{gid}')
def item_tile(gid):
    record=db.item(gid=gid)
    teaser=item_tile.track(item_teaser(gid=gid))
    return CachedResult(f'''
    [ {record.id} ]
    [ {record.title} ]
    [ {record.body} ]
    ''', (gid,))
print(item_tile('g0'))
print(item_tile('g3'))
print(item_tile('g4'))

@cached(db, key='page-{}-{}')
def page(start, stop):
    required_gids =[]
    for x in range(start, stop):
        cached_result = item_tile(f'g{x}')
        required_gids.extend(cached_result.required_gids)
    return CachedResult(f'Page for {start} to {stop}', required_gids)
print(page(0,2).value)
print(page(1,3).value)
print(page(0,4).value)



CachedResult(value='title 0: 0 0 0...', required_gids=['teaser-g0'])
CachedResult(value='title 2: 2 2 2...', required_gids=['teaser-g2'])
CachedResult(value='\n    [ 31 ]\n    [ title 0 ]\n    [ 0 0 0 0 0 ]\n    ', required_gids=['tile-g0'])
CachedResult(value='\n    [ 34 ]\n    [ title 3 ]\n    [ 3 3 3 3 3 ]\n    ', required_gids=['tile-g3'])
CachedResult(value='\n    [ 35 ]\n    [ title 4 ]\n    [ 4 4 4 4 4 ]\n    ', required_gids=['tile-g4'])
Page for 0 to 2
Page for 1 to 3
Page for 0 to 4


In [139]:
print('should be stable')
page(0,2)
page(1,3)
page(0,4)
print()
print('updating')
import random
db.item(gid='g0').update_record(title=f'testing2 {random.randint(1,100)}')
db.commit()
print()
print('should recalculate')
page(0,2) # Tile-G0 (g0) ,Tile-G1
page(1,3) # g1, g2,
page(0,4) # tile-g0, g1, g2, g3
print()
print('should be stable')
page(0,2)
page(1,3)
page(0,4)


should be stable

updating

should recalculate

should be stable


CachedResult(value='Page for 0 to 4', required_gids=('page-0-4',))

In [158]:
show('item',db(db.item).select())
show('cache',db(db.cache).select())
show('deps',db(db.deps).select())
print(debug_cache_dependencies(db))

----------------[ item ]----------------
|   id | gid   | title     | body      |
|------|-------|-----------|-----------|
|   32 | g1    | title 1   | 1 1 1 1 1 |
|   33 | g2    | title 2   | 2 2 2 2 2 |
|   34 | g3    | title 3   | 3 3 3 3 3 |
|   35 | g4    | title 4   | 4 4 4 4 4 |
|   31 | g0    | prut G011 | 0 0 0 0 0 |

---------------[ cache ]----------------
|   id | gid       | value             |
|------|-----------|-------------------|
|  213 | g0        |                   |
|  187 | g2        |                   |
|  188 | teaser-g2 | title 2: 2 2 2... |
|  190 | g3        |                   |
|  191 | teaser-g3 | title 3: 3 3 3... |
|  192 | tile-g3   | [ 34 ]
    [ title 3 ]
    [ 3 3 3 3 3 ]                   |
|  193 | g4        |                   |
|  194 | teaser-g4 | title 4: 4 4 4... |
|  195 | tile-g4   | [ 35 ]
    [ title 4 ]
    [ 4 4 4 4 4 ]                   |
|  196 | g1        |                   |
|  197 | teaser-g1 | title 1: 1 1 1... |
|  198 | tile-g

In [141]:
db.item(gid='g0').update_record(title='prut G011')
print(db._lastsql)
db.commit()

('UPDATE "item" SET "title"=\'prut G011\' WHERE ("item"."id" = 31);', 0.0011165142059326172)


In [160]:
show('item',db(db.item).select())
show('cache',db(db.cache).select())
show('deps',db(db.deps).select())

----------------[ item ]----------------
|   id | gid   | title     | body      |
|------|-------|-----------|-----------|
|   32 | g1    | title 1   | 1 1 1 1 1 |
|   33 | g2    | title 2   | 2 2 2 2 2 |
|   34 | g3    | title 3   | 3 3 3 3 3 |
|   35 | g4    | title 4   | 4 4 4 4 4 |
|   31 | g0    | prut G011 | 0 0 0 0 0 |

---------------[ cache ]----------------
|   id | gid       | value             |
|------|-----------|-------------------|
|  214 | g0        |                   |
|  187 | g2        |                   |
|  188 | teaser-g2 | title 2: 2 2 2... |
|  190 | g3        |                   |
|  191 | teaser-g3 | title 3: 3 3 3... |
|  192 | tile-g3   | [ 34 ]
    [ title 3 ]
    [ 3 3 3 3 3 ]                   |
|  193 | g4        |                   |
|  194 | teaser-g4 | title 4: 4 4 4... |
|  195 | tile-g4   | [ 35 ]
    [ title 4 ]
    [ 4 4 4 4 4 ]                   |
|  196 | g1        |                   |
|  197 | teaser-g1 | title 1: 1 1 1... |
|  198 | tile-g

In [152]:
db.commit()
cache_id = db(db.cache).select(db.cache.id, limitby=(0,1)).first().id
derivatives = db.executesql('select * from derivatives(%s)',placeholders=(cache_id,))
show_rows(f'derivatives of {cache_id}',derivatives)

[ derivatives of 187 ]
|   0 |
|-----|
| 188 |
| 187 |
| 201 |
| 200 |



In [147]:
derivatives

[(188,), (187,), (201,), (200,)]

In [161]:
item('g0').value
db.item(gid='g0').update_record(title='prut G011')
print(db._lastsql)
item('g0').value
print(db._lastsql)

item('g0').value

('UPDATE "item" SET "title"=\'prut G011\' WHERE ("item"."id" = 31);', 0.0010285377502441406)
('\n                    insert into deps (cache_id, depends_on)\n                SELECT 215, "cache"."id" FROM "cache" WHERE ("cache"."gid" IN (\'g0\'));', 0.00045800209045410156)


Item(id=31, gid='g0', title='prut G011', body='0 0 0 0 0')

In [162]:
dir(db)

['Field',
 'Row',
 'Rows',
 'Table',
 '_LAZY_TABLES',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapter',
 '_adapter_args',
 '_aliased_tables',
 '_attempts',
 '_bigint_id',
 '_check_reserved',
 '_common_fields',
 '_db_codec',
 '_db_uid',
 '_dbname',
 '_debug',
 '_decode_credentials',
 '_driver_args',
 '_drivers_available',
 '_fake_migrate',
 '_fake_migrate_all',
 '_folder',
 '_ignore_field_case',
 '_lastsql',
 '_lazy_tables',
 '_migrate',
 '_migrate_enabled',
 '_migrated',
 '_pending_references',
 '_pool_size',
 '_referee_name',
 '_remove_refe

In [163]:
db._timings

[('SELECT "cache"."id", "cache"."gid", "cache"."value" FROM "cache" WHERE ("cache"."gid" = \'page-0-4\') LIMIT 1 OFFSET 0;',
  0.00031495094299316406),
 ('SELECT "item"."id", "item"."gid", "item"."title", "item"."body" FROM "item" WHERE ("item"."id" IS NOT NULL);',
  0.00066375732421875),
 ('SELECT "cache"."id", "cache"."gid", "cache"."value" FROM "cache" WHERE ("cache"."id" IS NOT NULL);',
  0.00029540061950683594),
 ('SELECT "deps"."id", "deps"."cache_id", "deps"."depends_on" FROM "deps" WHERE ("deps"."id" IS NOT NULL);',
  0.0004029273986816406),
 ('\n        select cache.gid, dep.gid \n          from deps \n            inner join cache on cache.id = deps.cache_id \n            inner join cache dep on dep.id = deps.depends_on\n        ',
  0.0006132125854492188),
 ('SELECT "item"."id", "item"."gid", "item"."title", "item"."body" FROM "item" WHERE ("item"."gid" = \'g0\') LIMIT 1 OFFSET 0;',
  0.0007612705230712891),
 ('UPDATE "item" SET "title"=\'prut G011\' WHERE ("item"."id" = 31);